In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import spacy
spacy.load('en_core_web_sm')
# from epitator.annotator import AnnoDoc
# from epitator.geoname_annotator import GeonameAnnotator
# from epitator.resolved_keyword_annotator import ResolvedKeywordAnnotator
# from epitator.count_annotator import CountAnnotator
# from epitator.date_annotator import DateAnnotator
from itertools import groupby
import datetime
import sys
import pandas as pd
import re
import numpy as np
import epitator
import numpy as np

In [94]:
import pandas as pd
import pickle
import os
from who_scraper import scrape
from my_utils import extract_from_url, flatten_list
from annotator import *
from edb_clean import get_cleaned_edb, clean_disease_name
from wiki_country_parser import get_wiki_countries_df

In [152]:
y = pd.read_csv('diseaseCodes.csv', ';')
y.head()

,IdType,IdRecordType,IdParent,Code,TypeName,SqlTableName,CsName,GuiText,GuiTextShort,HasAccessRestriction,IsImmutable,UpdatedAt,ts
0,101,1,11,ADV,Adenovirus,Disease71ADV,Disease71ADV,Adenovirus,ADV,0,0,2016-10-28 07:39:22.327,0x00000000000E3525
1,102,1,11,BAN,Milzbrand,Disease71BAN,Disease71BAN,Milzbrand,BAN,0,0,2016-10-28 07:39:22.330,0x00000000000E3527
2,103,1,11,BOB,Borreliose,Disease71BOB,Disease71BOB,Lyme-Borreliose,BOB,0,0,2016-10-28 07:39:22.340,0x00000000000E3529
3,104,1,11,BOR,Läuserückfallfieber,Disease71BOR,Disease71BOR,Läuserückfallfieber,BOR,0,0,2016-10-28 07:39:22.340,0x00000000000E352B
4,105,1,11,BPS,Keuchhusten,Disease71BPS,Disease71BPS,Keuchhusten,BPS,0,0,2016-10-28 07:39:22.340,0x00000000000E352D


# WHO DF

In [ ]:
path = os.path.join("pickles","parsed_who_df.p")

In [ ]:
all_links = scrape(years=['2018'],proxies=None)
extracted = extract_from_url(all_links)

In [ ]:
parsed_whos_df = pd.DataFrame.from_dict(create_annotated_database(extracted, entity_funcs_and_params=[
                                                                      (geonames, {"raw":True}),
                                                                      (cases, {"raw":True}),
                                                                      (dates, {"raw":True}),
                                                                      keywords]))

In [ ]:
pickle.dump(parsed_whos_df,open(path,"wb"))

In [ ]:
parsed_whos_df = pickle.load(open(path,"rb"))
parsed_whos_df.head()

# EDB Statistics

In [111]:
edb = get_cleaned_edb(reduced=True)
edb.head()

,Ausgangs- bzw. Ausbruchsland,Sekundär betroffene Länder*,Krankheitsbild(er),Fälle gesamt*,Datenstand für Fallzahlen gesamt*,Fälle bestätigt,Warcheinlische Fälle,Verdachtsfälle,Todesfälle,Quelle 1*,...,Link zur Quelle 1,Quelle 2,Datum der Veröffentlichung der Quelle 2,Link zur Quelle 2,Quelle 3,Datum der Veröffentlichung der Quelle 3,Link zur Quelle 3,Quelle 4,Datum der Veröffentlichung der Quelle 4,Link zur Quelle 4
0,Nigeria,Benin,Lassafieber,1121,2018-03-18,376,NaN,NaN,119,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NCDC,2018-03-18,NaN,NaN,nan,NaN,NaN,nan,NaN
1,Benin,None,Lassafieber,24,2018-03-05,5,NaN,16,9,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
2,Liberia,None,Lassafieber,42,2018-03-16,7,NaN,NaN,14,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
3,Ghana,None,Lassafieber,1,2018-03-02,1,NaN,NaN,1,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
4,Fiji,None,Meningitis,18,nan,NaN,NaN,NaN,NaN,ProMED-mail,...,NaN,MoH Fiji,nan,NaN,NaN,nan,NaN,NaN,nan,NaN


In [143]:
clean_disease_name("Warzee, Gonorrhoee")

['wart', 'gonorrhea']

In [29]:
diseases = set(edb["Krankheitsbild(er)"].tolist())

In [30]:
translated = [clean_disease_name(i) for i in diseases]

# CONTINUE HERE

## Country statistic

In [144]:
print(len(edb)," entries", "in edb")
countries_not_null = edb[pd.notnull(edb["Ausgangs- bzw. Ausbruchsland"])]
print(len(countries_not_null)," are not empty")
unique = set(c[0] if type(c)==list else c for c in countries_not_null['Ausgangs- bzw. Ausbruchsland'].tolist())
print(len(unique), " are unique")

295  entries in edb
280  are not empty
80  are unique


In [145]:
wiki_countries_df = get_wiki_countries_df()
same_word_translated = []
for _, row in wiki_countries_df.iterrows():
    if row['state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['state_name_de'])
    elif row['full_state_name_de'] == row['translation_state_name']:
        same_word_translated.append(row['full_state_name_de'])
unprocesed_edb = pd.read_csv("edb.csv", sep=";")

# Exlude words that are unchanged compared to unproccessed version
unchanged_country_names = [e for e in unique if e in unprocesed_edb['Ausgangs- bzw. Ausbruchsland '].tolist()]
# Exlude words that are the same after translation
unchanged_country_names = [e for e in unchanged_country_names if e not in same_word_translated]
# Exlude words that were English in the first place
unchanged_country_names =  [e for e in unchanged_country_names if e not in wiki_countries_df["translation_state_name"].tolist()]
print("{} countries were not translated".format(len(unchanged_country_names)))
unchanged_country_names

8 countries were not translated


['Typhus',
 'VAE Dubai',
 'Französisch-Polynesien',
 'Nordeuropa',
 'La Reunion',
 'Französiche Guyana',
 'Taiwan',
 'DRCongo']

## Direct comparison to source

In [146]:
indices_sources = list(range(15,26))
indices_sources_and_countries = indices_sources
indices_sources_and_countries.extend([3,9,11,12])
indices_sources_and_countries

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 3, 9, 11, 12]

In [148]:
edb.head()

,Ausgangs- bzw. Ausbruchsland,Sekundär betroffene Länder*,Krankheitsbild(er),Fälle gesamt*,Datenstand für Fallzahlen gesamt*,Fälle bestätigt,Warcheinlische Fälle,Verdachtsfälle,Todesfälle,Quelle 1*,...,Link zur Quelle 1,Quelle 2,Datum der Veröffentlichung der Quelle 2,Link zur Quelle 2,Quelle 3,Datum der Veröffentlichung der Quelle 3,Link zur Quelle 3,Quelle 4,Datum der Veröffentlichung der Quelle 4,Link zur Quelle 4
0,Nigeria,Benin,Lassafieber,1121,2018-03-18,376,NaN,NaN,119,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NCDC,2018-03-18,NaN,NaN,nan,NaN,NaN,nan,NaN
1,Benin,None,Lassafieber,24,2018-03-05,5,NaN,16,9,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
2,Liberia,None,Lassafieber,42,2018-03-16,7,NaN,NaN,14,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
3,Ghana,None,Lassafieber,1,2018-03-02,1,NaN,NaN,1,WHO AFRO Bericht,...,<http://apps.who.int/iris/bitstream/10665/2604...,NaN,nan,NaN,NaN,nan,NaN,NaN,nan,NaN
4,Fiji,None,Meningitis,18,nan,NaN,NaN,NaN,NaN,ProMED-mail,...,NaN,MoH Fiji,nan,NaN,NaN,nan,NaN,NaN,nan,NaN


In [147]:
sources = edb.iloc[:,indices_sources_and_countries].copy(deep=True) # Get only the columns mentioning sources

IndexError: positional indexers are out-of-bounds

In [ ]:
mask = sources.copy(deep=True) # Create a mask for filtering
# Use only the columns about sources to filter
for column in sources.columns[:11]:
    mask[column] = sources[column].str.contains('who',na=False) # Extract all the entries that have the word "who"
    mask[column] = sources[column].str.contains('don',na=False) # and "don"
indices = [i for i in range(len(mask)) if not mask.iloc[i,:11].any()] 
sources_filtered = sources.drop(np.reshape(indices,(len(indices),)))# Drop all rows that don't mention "who" or "don"

In [ ]:
# Only get the necessary information from the who dons for comparison
comparable_who = parsed_whos_df.iloc[:,[1,2,4]].copy(deep=True)

In [ ]:
dates = sources_filtered.iloc[:,[1,4,7,10]].copy(deep=True)

In [ ]:
dates = dates.replace("nan",np.nan)

In [ ]:
# Clean dates
dates = dates.values.tolist()
dates = [list(filter(lambda x: x==x, row)) for row in dates]
dates

In [ ]:
# Clean faelle
faelle = sources_filtered.iloc[:,-3:].copy(deep=True)
faelle = faelle.replace(np.nan,"nan")
faelle = faelle.values.tolist()
faelle = [list(filter(lambda x: x==x, row)) for row in faelle]
faelle

In [ ]:
cases = []
for row in faelle:
    strings = []
    if len(row) >=1 :
        for string in row:
            string = str(string)
            found = re.findall(r"(\d{1,})",string)
            strings.extend([digit for digit in found if digit != '' and digit != "0"])
        cases.append(strings)
    else:
        cases.append(row)
cases

In [ ]:
sources_filtered.columns[-4]

In [ ]:
comparable_edb = sources_filtered['Ausgangs- bzw. Ausbruchsland'].copy(deep=True).to_frame()
comparable_edb["cases"] = cases
comparable_edb["dates"] = dates
comparable_edb

In [ ]:
comparable_who.head()

In [ ]:
# Match countries
found = 0
found_c = []
for state in comparable_edb["Ausgangs- bzw. Ausbruchsland"]:
    for i in comparable_who["geonames"]:
        if isinstance(state,str):
            if list(filter(lambda x: re.findall(state,x),i)):
                found_c.append(state)
                found += 1
                break

                    

In [ ]:
found_c

In [ ]:
[i for i in comparable_edb["Ausgangs- bzw. Ausbruchsland"] if i not in found_c]

In [ ]:
# Match dates
from my_utils import matching_elements
found_date = {}
for index, dates in enumerate(comparable_edb["dates"]):
    for i in comparable_who["dates"]:
        matches = matching_elements(dates,i)
        if len(matching_elemnts(dates,i)) >= 1:
            found_date[index] = matches        

In [ ]:
found_date

In [ ]:
found_cases = {}
for index, cases in enumerate(comparable_edb["cases"]):
    for i in comparable_who["cases"]:
        matches = matching_elements(cases,i)
        if len(matching_elemnts(dates,i)) >= 1:
            found_cases[index] = matches  

In [ ]:
comparable_edb["cases"]

In [ ]:
def find_matches(df1_and_indices, df2_and_indices):
    indices_matches = []
    matches = []
    df1, columns1 = df1_and_indices
    df2, columns2 = df2_and_indices
    for index , row in df1.iterrows():
        for _, row in df2.iterrows():
            if set()

In [ ]:
sources_filtered.columns[-2::-3]

In [ ]:
# Idea which is not correct and not complete
date_matches = {}
for column in sources_filtered.columns[-2::-3]: # Use only the columns mentioning dates
    date_matches[column] = [i for i in range(len(parsed_whos_df))\
                            if list(map(lambda x: str(x)[:-3],parsed_whos_df["date"].tolist()))[i] \
                            in list(map(lambda x: str(x)[:-12],sources_filtered[column].tolist()))]

In [ ]:
import itertools
indices_that_matched = list(set(itertools.chain(*date_matches.values())))

# To present

In [ ]:
ereignisdatenbank.iloc[sources_filtered.index.tolist(),[3,6,7,9]]

In [ ]:
parsed_whos_df.iloc[indices_that_matched,1:].sort_values("date")

In [ ]:
# Prettify the link description
link_description = [re.search(r'don/(.*)/en',all_links[i])[1]\
                    .replace('-', ' ',2).replace('-',', ',2).replace('-',' ')\
                    for i in range(len(all_links))]

In [ ]:
# Extract the most important columns
compare = parsed_whos_df.iloc[:,[1,3,4]].copy()
compare['link_description'] = pd.Series(link_description,index=compare.index)

In [ ]:
compare["date"].iloc[0]

In [ ]:
# These are the links that were faulty during annotation
to_check = compare[compare["keyword"].isnull()==True].index.values
links_to_check = np.asarray(all_links)[to_check] # Get the links that caused the bad annotations

In [ ]:
annotated_faulty_text = create_annotated_database(extract(links_to_check),raw=True)

In [ ]:
faulty_df = pd.DataFrame.from_dict(annotated_faulty_text).iloc[:,1:]
faulty_df

In [ ]:
# Entries that had the occurence of a geographical entity more than once
parsed_whos_df[parsed_whos_df["geoname"].str.len()>1]